# Particle Class

First, install `particula` from PyPI using `!pip install -U particula` or `%pip install -U particula`. The `!` are `%` symbols are so-called magic signs prefixing "magic functions". In this case, the magic function is `%pip` --- which is one way to install Python packages from the PyPI (the Python Package Index). The `-U` is to instruct `pip` to update particula to the latest version.

In [1]:
%pip install -U particula

     |████████████████████████████████| 209 kB 32.8 MB/s 


After installing `particula`, we can import it into our workspace with `import particula`. Let's check the version with `.__version__` and let's call `help` on the overall package to see what we have. 

In [2]:
import particula

In [3]:
particula.__version__

'0.0.3'

In [4]:
help(particula)

Help on package particula:

NAME
    particula - a simple, fast, and powerful particle simulator.

DESCRIPTION
    particula is a simple, fast, and powerful particle simulator,
    or at least two of the three, we hope. It is a simple particle
    system that is designed to be easy to use and easy to extend.
    The goal is to have a robust aerosol (gas phase + particle phase)
    simulation system that can be used to answer scientific questions
    that arise for experiments and research discussions.
    
    The main features of particula are:
    ...
    
    More details to follow.

PACKAGE CONTENTS
    aerosol_dynamics (package)

DATA
    u = <pint.registry.UnitRegistry object>

VERSION
    0.0.3

FILE
    /usr/local/lib/python3.7/dist-packages/particula/__init__.py




The above two code cells show that we have the prerelease version `0.0.3` (our latest as of writing this). We still are in the process of writing more thorough documentation, and so a help call on `particula` yields a minimal description. However, we get a hint about what else is inside this package, `aerosol_dynamics` package/module. We will check this now.

We also see `u = <pint.registry.UnitRegistry object>` under `DATA`: This is our unit registry which ensures proper unit tracking throughout each entire instance of the package. 

## `aerosol_dynamics` module

We can import `aerosol_dynamics` either via 

```python
from particula import aerosol_dynamics
```

or

```python
import particula.aerosol_dynamics
```

to check it out. We will call help again to see what we have. 

In [5]:
from particula import aerosol_dynamics

In [6]:
help(aerosol_dynamics)

Help on package particula.aerosol_dynamics in particula:

NAME
    particula.aerosol_dynamics - module to perform calculations related to aerosol dynamics.

DESCRIPTION
    This module contains functions to perform calculations related
    to aerosol dynamics. It inherits the unit registry, u, from
    particula. The pacakge contents below are callablle. For example,
    the particle module is callable as:
    
    >>> from particula.aerosol_dynamics import particle
    
    ...
    
    More details to follow.

PACKAGE CONTENTS
    environment
    parcel
    particle
    physical_parameters
    tests (package)

FILE
    /usr/local/lib/python3.7/dist-packages/particula/aerosol_dynamics/__init__.py




Now! We have some package contents that look promising. We will only check out `particle` in this case. Again, import it and call help on it.

In [7]:
from particula.aerosol_dynamics import particle

In [8]:
help(particle)

Help on module particula.aerosol_dynamics.particle in particula.aerosol_dynamics:

NAME
    particula.aerosol_dynamics.particle - to instantiate particles and calculate their properties.

DESCRIPTION
    This module contains the Particle class, which is used to
    instantiate the particles and calculate their properties.
    Particles are introduced and defined by calling Particle
    class, for example:
    
    >>> from particula import Particle
    >>> p1 = Particle(name='my_particle', radius=1e-9, density=1e3, charge=1)
    
    Then, it is possible to return the properties of the particle p1:
    
    >>> p1.mass()
    
    The environment is defined by the following parameters:
    >>> from particula.aerosol_dynamics import environment
    >>> env = environment.Environment(temperature=300, pressure=1e5)
    
    If another particle is introduced, it is possible to calculate
    the binary coagulation coefficient:
    
    >>> p2 = Particle(name='my_particle2', radius=1e-9, densi

We see there is more documentation now. The `help` call lists a "class" called "Particle" and all its available "methods" (read: functions) alphabetically. For now, you can think of a class as a super-function (it is more than that). Note also, that classes follow `CamelCase` (hence, `Particle`) whereas methods/functions follow snake_case (hence, `reduced_mass`). We make a lot of use of unit registry (called `u`) to ensure all our calculations have proper units, even if dimensionless. Without further ado, let's look at an example.

## Calculating coagulation 

Let's define to particles, `particle_a` and `particle_b`.

In [9]:
particle_a = particle.Particle(name='a', radius=1e-9, density=1e3, charge=0)

In [10]:
particle_b = particle.Particle(name='b', radius=2e-9, density=1e3, charge=0)

Then, if we call the `radius` method of particle_a (as suffix), we get its radius printed with a unit. Alternatively, we can call the `radius` method from the class itself, not prefixed to a certain particle (same result).

In [11]:
particle_a.radius()

1e-09 meter

In [12]:
particle.Particle.radius(particle_a)

1e-09 meter

Same for the `mass` and `charge` methods. Note `charge` here is the number of charges. 

In [13]:
particle_b.mass()

3.3510321638291136e-23 kilogram

In [14]:
particle_a.charge()

0 dimensionless

We should also define the environment we are in. We will get that module and invoke with 300 K and 10,000 Pa. 

In [15]:
from particula.aerosol_dynamics import environment

In [16]:
std_environment = environment.Environment(temperature=300, pressure=1e5)

Now, we can calculate the Knudsen number of particle "a" in this "standard" environment. And the slip correction factor. And the friction factor. 

In [17]:
particle_a.knudsen_number(std_environment)

67.8918099704083 dimensionless

In [18]:
particle_a.slip_correction_factor(std_environment)

113.060274442495 dimensionless

In [19]:
particle_a.friction_factor(std_environment)

3.0775355689697488e-15 kilogram/second

Now, let's calculate the reduced mass of two particles, "a" and "b". Again, there are two ways to do this: by suffixing the method to a particle or by calling the method from the class --- they yield the same result. 

In [20]:
particle_a.reduced_mass(particle_b)

3.7233690709212374e-24 kilogram

In [21]:
particle.Particle.reduced_mass(particle_a, particle_b)

3.7233690709212374e-24 kilogram

Like the reduced mass, we can also look at the reduced friction. 

In [22]:
particle_a.reduced_friction_factor(particle_b, std_environment)

2.4595460300833962e-15 kilogram/second

We can also calculate the coagulation rate coefficient between particles "a" and "b" in this environment. Note that we could have called this directly after defining "a" and "b".

In [23]:
particle.Particle.dimensionless_coagulation_kernel_hard_sphere(
    particle_a, particle_b, std_environment
)

83.4628646051075 dimensionless

In [24]:
particle_a.dimensionless_coagulation_kernel_hard_sphere(
    particle_b, std_environment
)

83.4628646051075 dimensionless

## The coagulation rate coefficient

Perhaps most importantly, we can calculate the binary coagulation rate coefficient. This is done via the method `dimensioned_coagulation_kernel`. Let's call help on this method to see what we can do.

In [25]:
help(particle.Particle.dimensioned_coagulation_kernel)

Help on function dimensioned_coagulation_kernel in module particula.aerosol_dynamics.particle:

dimensioned_coagulation_kernel(self, other, environment: particula.aerosol_dynamics.environment.Environment, authors: str = 'cg2019') -> float
    Dimensioned particle--particle coagulation kernel.
    
    Checks units: [m**3/s]
    
    Paramaters:
    
        self:           particle 1
        other:          particle 2
        environment:    environment conditions
        authors:        authors of the parameterization
            - gh2012    https://doi.org/10.1103/PhysRevE.78.046402
            - cg2020    https://doi.org/XXXXXXXXXXXXXXXXXXXXXXXXXX
            - hard_sphere (from above)
            (default: cg2019)



We see that we have have to supply two particles alongside the environment --- additionally, we get to choose between different parameterizations under "authors", which defaults to a recent study.

In [26]:
particle_a.dimensioned_coagulation_kernel(
    particle_b, std_environment
)

4.655713855465682e-16 meter³/second

For a more interesting example, let's look at the interaction between a negative ion and a positive particle. We first need to define these two entities alongside an environment.

In [27]:
negative_ion = particle.Particle(
    name="negative_ion",
    radius=0.5e-9,
    density=1.84e3,
    charge=-1,
)

positive_particle = particle.Particle(
    name="positive_particle",
    radius=3e-9,
    density=1.7e3,
    charge=1,
)

standard_environment_ip = environment.Environment(
    temperature=300,
    pressure=101325,
)

The coagulation between these two entities should be approximately $10^{-6}$ cm$^{3}$/s.

In [28]:
negative_ion.dimensioned_coagulation_kernel(
    positive_particle, standard_environment_ip
)

4.670176867752661e-13 meter³/second

That is approximatley $5 \times 10^{-7}$ cm$^{3}$/s, which is within an order of magnitude of our physics-based reasoning.